In [111]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, DateTime, func, Text, select
from sqlalchemy import Float, Boolean
from sqlalchemy.orm import declarative_base, Session, relationship, sessionmaker, configure_mappers
from datetime import datetime, timezone
from typing import List, Tuple, Dict
import pandas as pd
from IPython.display import display, HTML
from typing import List, Tuple, Dict

# Loading the dataframe

In [2]:
# Try to determine the encoding of the dataframe

import chardet

with open('content/globalterrorismdb_0718dist.csv', 'rb') as f:
    result = chardet.detect(f.read(100000))  # Read the first 100000 bytes
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [3]:
df_original = pd.read_csv('content/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1', low_memory=False)

In [4]:
# Get base infomation from dataframe
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


In [5]:
df_original


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN


In [110]:
# Check for unique data and systematize it

from typing import List

df = df_original.copy()

for col in df.columns:
    unique_count: int = df[col].nunique()  # Number of unique values for each column
    unique_values: List = df[col].unique().tolist()  # List of unique values for each column
    
    # Check if all values can be converted to numbers (to filter out numeric fields)
    is_numeric: bool = pd.to_numeric(df[col], errors='coerce').notnull().all()
    
    if is_numeric:
        print(f"Column '{col}': unique values {unique_count} - numeric values")
    else:
        print(f"Column '{col}': unique values {unique_count} — {unique_values[:5]}")


Column 'eventid': unique values 181691 - numeric values
Column 'iyear': unique values 47 - numeric values
Column 'imonth': unique values 13 - numeric values
Column 'iday': unique values 32 - numeric values
Column 'approxdate': unique values 2244 — [nan, 'January 19-20, 1970', 'February 6-9, 1970', 'February 11-14, 1970', 'February 16-17, 1970']
Column 'extended': unique values 2 - numeric values
Column 'resolution': unique values 1859 — [nan, '3/8/1970', '3/15/1970', '3/31/1970', '3/26/1970']
Column 'country': unique values 205 - numeric values
Column 'country_txt': unique values 205 — ['Dominican Republic', 'Mexico', 'Philippines', 'Greece', 'Japan']
Column 'region': unique values 12 - numeric values
Column 'region_txt': unique values 12 — ['Central America & Caribbean', 'North America', 'Southeast Asia', 'Western Europe', 'East Asia']
Column 'provstate': unique values 2855 — [nan, 'Federal', 'Tarlac', 'Attica', 'Fukouka']
Column 'city': unique values 36673 — ['Santo Domingo', 'Mexico

# Create DataBase

## Create tables

In [85]:
Base = declarative_base()

class Incident(Base):
    
    """
    Basic table "Incident":
    
    Columns:
    Columns:
    iyear - Year when the incident occurred
    imonth - Month when the incident occurred (1-12, 0 if unknown)
    iday - Day when the incident occurred (1-31, 0 if unknown)
    city_id - Reference to the city where the incident took place
    type_incident_id - Type of the incident (e.g., bombing, armed assault)
    type_attack_id - Method of attack used in the incident
    target_sub_type_id - Specific type of target affected by the incident
    corp_id - ID of the corporation or organization involved (if applicable)
    target_id - Reference to the main target of the incident
    natlty_id - Nationality of the target affected by the incident
    claimmode_id - Mode of claim responsibility (e.g., official statement, media)
    weapon_sub_type_id - Specific type of weapon used in the incident
    propextent_id - Extent of property damage caused by the incident
    """
    __tablename__ = 'incident'
    
    id = Column(Integer, primary_key=True)  # eventid
    iyear = Column(Integer, nullable=False)
    imonth = Column(Integer, nullable=False)
    iday = Column(Integer, nullable=False)
    city_id = Column(Integer, ForeignKey('city.id'))
    type_incident_id = Column(Integer, ForeignKey('type_incident.id'))
    type_attack_id = Column(Integer, ForeignKey('type_attack.id'))
    target_sub_type_id = Column(Integer, ForeignKey('target_sub_type.id'))
    corp_id = Column(Integer, ForeignKey('corp.id'))
    target_id = Column(Integer, ForeignKey('target.id'))
    natlty_id = Column(Integer, ForeignKey('natlty.id'))
    claimmode_id = Column(Integer, ForeignKey('claimmode.id'))
    weapon_sub_type_id = Column(Integer, ForeignKey('weapon_sub_type.id')) 
    propextent_id = Column(Integer, ForeignKey('propextent.id'))

    city = relationship(lambda:City, back_populates="incident")
    place_spec = relationship(lambda:PlaceSpec, back_populates="incident", uselist=False)
    attack_spec = relationship(lambda:AttackSpec, back_populates="incident", uselist=False)
    type_incident = relationship(lambda:Type_Incident, back_populates="incident")
    type_attack = relationship(lambda:Type_Attack, back_populates="incident")
    target_sub_type = relationship(lambda:Target_Sub_Type, back_populates="incident")
    corp = relationship(lambda:Corp, back_populates="incident")
    target = relationship(lambda:Target, back_populates="incident")
    natlty = relationship(lambda:Natlty, back_populates="incident")
    claimmode = relationship(lambda:Claimmode, back_populates="incident") 
    weapon_sub_type = relationship(lambda:Weapon_Sub_Type, back_populates="incident") 
    propextent = relationship(lambda:Propextent, back_populates="incident") 
    victims = relationship(lambda:Victims, back_populates="incident")
    properties = relationship(lambda:Properties, back_populates="incident")
    hostages = relationship(lambda:Hostages, back_populates="incident")
    international_data = relationship(lambda:International_Data, back_populates="incident")
    other_data = relationship(lambda:Other_Data, back_populates="incident")


class Country(Base):

    """
    table "Country":

    Columns:
    country_txt - name of the country
    """
    
    __tablename__ = 'country'
    id = Column(Integer, primary_key=True)
    country_txt = Column(String, nullable=False)

    region = relationship(lambda:Region, back_populates="country")

class Region(Base):
    
    """
    table "Region":

    Columns:
    region_txt - name of the region
    country_id - id country
    """
    
    __tablename__ = 'region'
    id = Column(Integer, primary_key=True)
    region_txt = Column(String, nullable=False)
    country_id = Column(Integer, ForeignKey('country.id'))

    city = relationship(lambda:City, back_populates="region")
    country = relationship(lambda:Country, back_populates="region")

class City(Base):
    
    """
    table "City":

    Columns:
    city - name of the city
    region_id - id region
    """
    
    __tablename__ = 'city'
    id = Column(Integer, primary_key=True)
    city = Column(String, nullable=False)
    region_id = Column(Integer, ForeignKey('region.id'))

    incident = relationship(lambda:Incident, back_populates="city")
    region = relationship(lambda:Region, back_populates="city")

class PlaceSpec(Base):

    """
    table "PlaceSpec":

    Columns:
    latitude - latitude
    longitude - longitude
    specificity -  Level of specificity of the location
    vicinity - Whether the attack occurred in the immediate vicinity (True/False/Unknown)
    location - Detailed location description
    summary - Brief summary of the incident
    motive - tated or suspected motive for the attack
    incident_id -  id in base table Incident
    """

    __tablename__ = 'place_spec'
    id = Column(Integer, primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    specificity = Column(Integer)
    vicinity = Column(Boolean, nullable=True)  # Допускаем NaN из CSV
    location = Column(String)
    summary = Column(Text)
    motive = Column(Text)

    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="place_spec")

class AttackSpec(Base):

    """
    table "AttackSpec":

    Columns:
    crit1 - Political, economic, religious, or social goal
    crit2 - Evidence of intention to coerce, intimidate, or convey a message
    crit3 -  Action outside the context of legitimate warfare
    doubtterr - Whether there is doubt that the incident was terrorism
    multiple - Whether multiple attacks were part of the same incident
    success - Whether the attack was successful
    suicide - Whether it was a suicide attack
    incident_id -  id in base table Incident
    """

    __tablename__ = 'attack_spec'
    id = Column(Integer, primary_key=True)
    crit1 = Column(Boolean, nullable=True)
    crit2 = Column(Boolean, nullable=True)
    crit3 = Column(Boolean, nullable=True)
    doubtterr = Column(Boolean, nullable=True)
    multiple = Column(Boolean, nullable=True)
    success = Column(Boolean, nullable=True)
    suicide = Column(Boolean, nullable=True)

    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="attack_spec")

class Victims(Base):

    """
    table "Victims":

    Columns:
    nkill - Total number of people killed
    nkillus - number of U.S. citizens killed
    nkillter -  Number of perpetrators killed
    nwound - Total number of people wounded
    nwoundus - Number of U.S. citizens wounded
    nwoundte - Number of perpetrators wounded
    incident_id -  id in base table Incident
    """

    __tablename__ = 'victims'
    id = Column(Integer, primary_key=True)
    nkill = Column(Integer)
    nkillus = Column(Integer)
    nkillter = Column(Integer)
    nwound = Column(Integer)
    nwoundus = Column(Integer)
    nwoundte = Column(Integer)
    
    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="victims")

class Properties(Base):

    """
    table "Properties":

    Columns:
    property_ - Whether property damage occurred
    propvalue - Estimated property damage value
    propcomment -  Additional comments on property damage
    incident_id -  id in base table Incident
    """
    
    __tablename__ = 'properties'
    id = Column(Integer, primary_key=True)
    property_ = Column(Boolean, nullable=False)
    propvalue = Column(Integer, nullable=False)
    propcomment = Column(Text, nullable=False)

    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="properties")

class Hostages(Base):
    
    """
    table "Hostages":

    Columns:
    ishostkid - Whether hostages/kidnapping occurred
    nhostkid - Number of hostages/kidnapped victims
    nhostkidus -  Number of U.S. hostages/kidnapped victims
    ndays - Duration of hostage situation (days)
    ransom - Whether ransom was demanded
    ransomamt - Amount of ransom demanded
    ransompaid - Amount of ransom paid
    hostkidoutcome_txt - Outcome of the hostage/kidnapping event
    incident_id -  id in base table Incident
    """
    
    __tablename__ = 'hostages'
    id = Column(Integer, primary_key=True)
    ishostkid = Column(Boolean, nullable=False)
    nhostkid = Column(Integer)
    nhostkidus = Column(Integer)
    ndays = Column(Integer)
    ransom = Column(Boolean)
    ransomamt = Column(Integer)
    ransompaid = Column(Integer)
    hostkidoutcome_txt = Column(String, nullable=False)

    incident_id = Column(Integer, ForeignKey('incident.id'))

    incident = relationship(lambda:Incident, back_populates="hostages")

class International_Data(Base):

    """
    table "International_Data":

    Columns:
    INT_LOG - International logistically-related terrorism indicator
    INT_IDEO - International ideological-related terrorism indicator
    INT_MISC -  International miscellaneous terrorism indicator
    INT_ANY - Whether any international connection exists
    incident_id -  id in base table Incident
    """
    
    __tablename__ = 'international_data'
    id = Column(Integer, primary_key=True)
    INT_LOG = Column(Boolean, nullable=False)
    INT_IDEO = Column(Boolean, nullable=False)
    INT_MISC = Column(Boolean, nullable=False)
    INT_ANY = Column(Boolean, nullable=False)

    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="international_data")

class Type_Incident(Base):

    """
    table "Type_Incident":

    Columns:
    alternative_txt - Alternative description of the incident
    """
    
    __tablename__ = 'type_incident'
    
    id = Column(Integer, primary_key=True)   # заполняется из alternative
    alternative_txt = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="type_incident")
    
class Type_Attack(Base):
    
    """
    table "Type_Incident":

    Columns:
    attacktype1_txt - Primary attack type description
    """
    
    __tablename__ = 'type_attack'
    
    id = Column(Integer, primary_key=True)   # заполняется из attacktype1
    attacktype1_txt = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="type_attack")

class Target_Type(Base):

    """
    table "Target_Type":

    Columns:
    targtype1_txt - General target type description
    """
    
    __tablename__ = 'target_type'
    
    id = Column(Integer, primary_key=True)   # заполняется из targtype1
    targtype1_txt = Column(String, nullable=False)

    target_sub_type = relationship(lambda:Target_Sub_Type, back_populates="target_type")


class Target_Sub_Type(Base):

    """
    table "Target_Sub_Type":

    Columns:
    targsubtype1_txt - Specific target subtype description
    target_type_id - id in Target_Type
    """
    
    __tablename__ = 'target_sub_type'
    
    id = Column(Integer, primary_key=True)   # заполняется из targsubtype1
    targsubtype1_txt = Column(String, nullable=False)

    target_type_id = Column(Integer, ForeignKey('target_type.id'))
    target_type = relationship(lambda:Target_Type, back_populates="target_sub_type")
    incident = relationship(lambda:Incident, back_populates="target_sub_type")

class Corp(Base):

    """
    table "Corp":

    Columns:
    corp1 - Name of the corporation or organization targeted
    """
    
    __tablename__ = 'corp'
    
    id = Column(Integer, primary_key=True)   # cвои
    corp1 = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="corp")

class Target(Base):
    
    """
    table "Target":

    Columns:
    corp1 - Specific name of the target
    """
    
    __tablename__ = 'target'
    
    id = Column(Integer, primary_key=True)   # cвои
    target1 = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="target")

class Natlty(Base):

    """
    table "Natlty":

    Columns:
    natlty1_txt - Nationality of the target
    """
    
    __tablename__ = 'natlty'
    
    id = Column(Integer, primary_key=True)   # заполняется из natlty1
    natlty1_txt = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="natlty")

class Claimmode(Base):

    """
    table "Claimmode":

    Columns:
    claimmode_txt - Mode of claim responsibility description
    """
    
    __tablename__ = 'claimmode'
    
    id = Column(Integer, primary_key=True)   # заполняется из claimmode
    claimmode_txt = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="claimmode")

class Weapon_Type(Base):

    """
    table "Weapon_Type":

    Columns:
    weaptype1_txt - General weapon type description
    """
    
    __tablename__ = 'weapon_type'
    
    id = Column(Integer, primary_key=True)   # заполняется из weaptype1
    weaptype1_txt = Column(String, nullable=False)

    weapon_sub_type = relationship(lambda:Weapon_Sub_Type, back_populates="weapon_type")
    
class Weapon_Sub_Type(Base):

    """
    table "Weapon_Sub_Type":

    Columns:
    weapsubtype1_txt - Specific weapon subtype description
    weapon_type_id - id in Weapon_Type
    """
    
    __tablename__ = 'weapon_sub_type'
    
    id = Column(Integer, primary_key=True)   # заполняется из weapsubtype1
    weapsubtype1_txt = Column(String, nullable=False)

    weapon_type_id = Column(Integer, ForeignKey('weapon_type.id'))
    weapon_type = relationship(lambda:Weapon_Type, back_populates="weapon_sub_type")
    incident = relationship(lambda:Incident, back_populates="weapon_sub_type")

class Propextent(Base):

    """
    table "Propextent":

    Columns:
    propextent_txt - Description of property damage extent
    """
    
    __tablename__ = 'propextent'
    
    id = Column(Integer, primary_key=True)   # свои
    propextent_txt = Column(String, nullable=False)

    incident = relationship(lambda:Incident, back_populates="propextent")

class Other_Data(Base):

    """
    table "Other_Data":

    Columns:
    addnotes - Additional notes on the incident
    scite1 - First source citation
    scite2 - Second source citation
    scite3 - Third source citation
    incident_id -  id in base table Incident
    """

    __tablename__ = 'other_data'

    id = Column(Integer, primary_key=True)   # свои
    addnotes = Column(Text, nullable=False)
    scite1 = Column(Text, nullable=False)
    scite2 = Column(Text, nullable=False)
    scite3 = Column(Text, nullable=False)

    incident_id = Column(Integer, ForeignKey('incident.id'))
    incident = relationship(lambda:Incident, back_populates="other_data")

engine = create_engine("sqlite:///content/incident.db")



In [86]:
# Execute once to create the database file. After that, do not execute the line
# Base.metadata.create_all(engine)

In [30]:
# Function to check if everything is OK with the relationship in the tables (if it doesn't give an error then everything is fine)
# configure_mappers()

## Create class ORM 

In [87]:
# for use class methods to fill the table in DB

Session = sessionmaker(bind=engine)

class ORM:

    @staticmethod
    def fill_countries():
        
        """Method.Filling in the table of countries"""
        
        with Session() as session:
            
            unique_countries: pd.DataFrame = df[['country', 'country_txt']].drop_duplicates()
            
            country_data: List[Dict[str, int|str]] = [
                {"id": row["country"], "country_txt": row["country_txt"]}
                for _, row in unique_countries.iterrows()
            ]
            session.bulk_insert_mappings(Country, country_data)
            session.commit()

    @staticmethod
    def fill_regions():
        
        """Method.Filling in the table of region, linking to country"""
        
        with Session() as session:
            
           # Load existing countries into dictionary {country id: country id}
            country_ids: set[int] = {c.id: c.id for c in session.query(Country).all()}
    
           # Remove duplicates by region, leaving country (country id)
            unique_regions: pd.DataFrame = df[['region', 'region_txt', 'country']].drop_duplicates(subset=['region'])
    
            region_data: List[Dict[str, int|str]] = [
                {
                    "id": row["region"],
                    "region_txt": row["region_txt"],
                    "country_id": country_ids.get(row["country"])  # country - it' id
                }
                for _, row in unique_regions.iterrows()
                if row["country"] in country_ids  
            ]
    
   
            session.bulk_insert_mappings(Region, region_data)
            session.commit()

    @staticmethod
    def fill_cities():
        
        """Method. Filling the table of cities, linking to regions"""
        
        with Session() as session:
            
            # Load regions into dictionary {region id: region id}
            region_ids: set[int] = {r.id: r.id for r in session.scalars(select(Region))}
    
            # Filter unique cities, remove empty and NaN
            unique_cities: pd.DataFrame = df[['city', 'region']].drop_duplicates()
            unique_cities = unique_cities.dropna(subset=['city'])  # Remove NaN in 'city'
            unique_cities = unique_cities[unique_cities['city'].str.strip() != ""]  # Remove empty lines
    
            city_data: List[Dict[str, int|str]] = [
                {
                    "city": row["city"],
                    "region_id": region_ids.get(row["region"])
                }
                for _, row in unique_cities.iterrows()
                if row["region"] in region_ids  # Exclude non-existent regions
            ]
    
   
            session.bulk_insert_mappings(City, city_data)
            session.commit()

    @staticmethod
    def fill_type_incident():
        
        """Method. Filling the table of  type_incident"""
        
        with Session() as session:
            
            # remove empty and NaN
            unique_types: pd.DataFrame = df[['alternative', 'alternative_txt']].dropna().drop_duplicates()
    
            # Convert to a list of dictionaries by casting 'alternative' to int
            type_incident_data: List[Dict[str, int|str]] = [
                {"id": int(row["alternative"]), "alternative_txt": row["alternative_txt"]}
                for _, row in unique_types.iterrows()
            ]
    
    
            session.bulk_insert_mappings(Type_Incident, type_incident_data)
            session.commit()

    @staticmethod
    def fill_type_attack():
        
        """Method. Filling the table of type_attack"""
        
        with Session() as session:
            
            # remove empty and NaN
            unique_types: pd.DataFrame = df[['attacktype1', 'attacktype1_txt']].dropna().drop_duplicates()
    
             # Convert to a list of dictionaries by casting 'attacktype1' to int
            type_attack_data: List[Dict[str, int|str]] = [
                {"id": int(row["attacktype1"]), "attacktype1_txt": row["attacktype1_txt"]}
                for _, row in unique_types.iterrows()
            ]
    
   
            session.bulk_insert_mappings(Type_Attack, type_attack_data)
            session.commit()

    @staticmethod
    def fill_target_type():
        
        """Method. Filling the table of target_type"""
        
        with Session() as session:
            
            unique_data: pd.DataFrame = df[['targtype1', 'targtype1_txt']].drop_duplicates()
            
            all_data: List[Dict[str, int|str]] = [
                {"id": row["targtype1"], "targtype1_txt": row["targtype1_txt"]}
                for _, row in unique_data.iterrows()
            ]
            session.bulk_insert_mappings(Target_Type, all_data)
            session.commit()

    @staticmethod
    def fill_target_sub_type():
        
        """Method. Filling the table of fill_target_sub_type"""
        
        with Session() as session:
            
            # Load existing target_type into dictionary {id target_type: id target_type}
            target_type_ids: set[int] = {t.id: t.id for t in session.query(Target_Type).all()}

            unique_data: pd.DataFrame = df[['targsubtype1', 'targsubtype1_txt', 'targtype1']].dropna(subset=['targsubtype1_txt']).drop_duplicates(subset=['targsubtype1'])
    
            # Form data into a list of dictionaries
            all_data: List[Dict[str, int|str]] = [
                {
                    "id": int(row["targsubtype1"]),  
                    "targsubtype1_txt": row["targsubtype1_txt"],
                    "target_type_id": target_type_ids.get(int(row["targtype1"]))  # targtype1 - это id
                }
                for _, row in unique_data.iterrows()
                if int(row["targtype1"]) in target_type_ids  
            ]
    
            print("Data to insert:", all_data[:10])  # Check first 10 elements
    
            session.bulk_insert_mappings(Target_Sub_Type, all_data)
            session.commit()


    @staticmethod
    def fill_corp():
        
        """Method. Filling the table of corp"""
        
        with Session() as session:
            
            unique_types: pd.DataFrame = df[['corp1']].dropna().drop_duplicates()
    
            corp_data: List[Dict[str, int|str]] = [
                {"corp1": row["corp1"]}
                for _, row in unique_types.iterrows()
            ]
    
            print("Data to insert:", corp_data[:10])  
            session.bulk_insert_mappings(Corp, corp_data)
            session.commit()

    @staticmethod
    def fill_target():
        
        """Method. Filling the table of target"""
        
        with Session() as session:
           
            unique_types: pd.DataFrame = df[['target1']].dropna().drop_duplicates()
    
            target_data: List[Dict[str, int|str]] = [
                {"target1": row["target1"]}
                for _, row in unique_types.iterrows()
            ]
    
            print("Data to insert:", target_data[:10]) 
            session.bulk_insert_mappings(Target, target_data)
            session.commit()

    @staticmethod
    def fill_natlty():
        
        """Method. Filling the table of natlty"""
        
        with Session() as session:
            
            unique_types: pd.DataFrame = df[['natlty1', 'natlty1_txt']].dropna().drop_duplicates()
    
            natlty_data: List[Dict[str, int|str]] = [
                {"id": int(row["natlty1"]), "natlty1_txt": row["natlty1_txt"]}
                for _, row in unique_types.iterrows()
            ]
            print("Data to insert:", natlty_data[:10]) 
   
            session.bulk_insert_mappings(Natlty, natlty_data)
            session.commit()

    @staticmethod
    def fill_claimmode():
        
        """Method. Filling the table of claimmode"""
        
        with Session() as session:
           
            unique_types: pd.DataFrame = df[['claimmode', 'claimmode_txt']].dropna().drop_duplicates()
    
            claimmode_data: List[Dict[str, int|str]] = [
                {"id": int(row["claimmode"]), "claimmode_txt": row["claimmode_txt"]}
                for _, row in unique_types.iterrows()
            ]
            print("Data to insert:", claimmode_data[:10])  
   
            session.bulk_insert_mappings(Claimmode, claimmode_data)
            session.commit()



    @staticmethod
    def fill_weapon_type():
        
        """Method. Filling the table weapon_type"""
        
        with Session() as session:
            
            unique_data: pd.DataFrame = df[['weaptype1', 'weaptype1_txt']].drop_duplicates()
            
            all_dataa: List[Dict[str, int|str]] = [
                {"id": row["weaptype1"], "weaptype1_txt": row["weaptype1_txt"]}
                for _, row in unique_data.iterrows()
            ]

            print("Data to insert:", all_data[:10])  
            session.bulk_insert_mappings(Weapon_Type, all_data)
            session.commit()

    @staticmethod
    def fill_weapon_sub_type():
        
        """Method. Filling the table weapon_sub_type"""
        
        with Session() as session:
            
            weapon_sub_type_ids: set[int] = {t.id: t.id for t in session.query(Weapon_Type).all()}
    
            unique_data: pd.DataFrame = df[['weapsubtype1', 'weapsubtype1_txt', 'weaptype1']].dropna(subset=['weapsubtype1_txt']).drop_duplicates(subset=['weapsubtype1'])
    
            all_data: List[Dict[str, int|str]] = [
                {
                    "id": int(row["weapsubtype1"]), 
                    "weapsubtype1_txt": row["weapsubtype1_txt"],
                    "weapon_type_id": weapon_sub_type_ids.get(int(row["weaptype1"]))  # weaptype1 - it's id
                }
                for _, row in unique_data.iterrows()
                if int(row["weaptype1"]) in weapon_sub_type_ids  #
            ]
    
            print("Data to insert:", all_data[:10])  #
    
            session.bulk_insert_mappings(Weapon_Sub_Type, all_data)
            session.commit()


    @staticmethod
    def fill_propextent():
        
        """Method. Filling the table propextent"""
        
        with Session() as session:
            
            unique_data: pd.DataFrame = df[['propextent', 'propextent_txt']].dropna().drop_duplicates()  
    
            all_data: List[Dict[str, int|str]] = [
                {"id": int(row["propextent"]), "propextent_txt": row["propextent_txt"]}
                for _, row in unique_data.iterrows()
                if not pd.isna(row["propextent"]) and not pd.isna(row["propextent_txt"])  # Filter NaN
            ]

            print("Data to insert:", all_data[:10])  
            session.bulk_insert_mappings(Propextent, all_data)
            session.commit()

    @staticmethod
    def fill_incident():
        
        """Method. Filling the table incident"""
        
        with Session() as session:
            
            # Load IDs of related tables
            city_ids: set[int] = {c.city: c.id for c in session.query(City).all()}
            alternative_ids: set[int] = {t.id: t.id for t in session.query(Type_Incident).all()}
            type_attack_ids: set[int] = {t.id: t.id for t in session.query(Type_Attack).all()}
            target_sub_type_ids: set[int] = {t.id: t.id for t in session.query(Target_Sub_Type).all()}
            corp_ids: set[int] = {c.corp1: c.id for c in session.query(Corp).all()}
            target_ids: set[int] = {t.target1: t.id for t in session.query(Target).all()}
            natlty_ids: set[int] = {n.id: n.id for n in session.query(Natlty).all()}
            claimmode_ids: set[int] = {c.id: c.id for c in session.query(Claimmode).all()}
            weapon_sub_type_ids: set[int] = {w.id: w.id for w in session.query(Weapon_Sub_Type).all()}
            propextent_ids: set[int] = {p.id: p.id for p in session.query(Propextent).all()}
    
            df_clean: pd.DataFrame = df[['eventid', 'iyear', 'imonth', 'iday', 'city', 'alternative', 'attacktype1', 
                           'targsubtype1', 'corp1', 'target1', 'natlty1', 'claimmode', 
                           'weapsubtype1', 'propextent']].drop_duplicates().where(pd.notna(df), None)
    
            all_data: List[Dict[str, int]]  = []
            for _, row in df_clean.iterrows():
                incident_data = {
                    "id": int(row["eventid"]),
                    "iyear": int(row["iyear"]),
                    "imonth": int(row["imonth"]),
                    "iday": int(row["iday"]),
                    "city_id": city_ids.get(row["city"]),  # Now we search for ID by city name
                    "type_incident_id": alternative_ids.get(row["alternative"]),
                    "type_attack_id": type_attack_ids.get(row["attacktype1"]),
                    "target_sub_type_id": target_sub_type_ids.get(row["targsubtype1"]),
                    "corp_id": corp_ids.get(row["corp1"]),  # Now we search for ID by corp1 name
                    "target_id": target_ids.get(row["target1"]),
                    "natlty_id": natlty_ids.get(row["natlty1"]),
                    "claimmode_id": claimmode_ids.get(row["claimmode"]),
                    "weapon_sub_type_id": weapon_sub_type_ids.get(row["weapsubtype1"]),
                    "propextent_id": propextent_ids.get(row["propextent"]),
                }
                all_data.append(incident_data)
    
            session.bulk_insert_mappings(Incident, all_data)
            session.commit()

    @staticmethod
    def fill_other_data():
        
        """Method. Filling the table other_data"""
        
        with Session() as session:

            incident_ids: set[int]  = {i.id for i in session.query(Incident.id).all()}
    
            # Filter the source data, remove lines without incident_id
            filtered_df: pd.DataFrame = df[['eventid', 'addnotes', 'scite1', 'scite2', 'scite3']].dropna(subset=['eventid'])
    
            other_data_entries: List[Dict[str, int|str]] = [
                {
                    "id": int(row["eventid"]),  # ID matches eventid from incident
                    "addnotes": row["addnotes"] if pd.notna(row["addnotes"]) else "",
                    "scite1": row["scite1"] if pd.notna(row["scite1"]) else "",
                    "scite2": row["scite2"] if pd.notna(row["scite2"]) else "",
                    "scite3": row["scite3"] if pd.notna(row["scite3"]) else "",
                    "incident_id": int(row["eventid"])  # incident_id = eventid
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids  # Only those in Incident
            ]
            print("Data to insert:", other_data_entries[:10])  
            session.bulk_insert_mappings(Other_Data, other_data_entries)
            session.commit()

    @staticmethod
    def fill_place_spec():  
        
        """Method. Filling the table place_spec"""
        
        with Session() as session:
            
            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}
    
            filtered_df: pd.DataFrame = df[['eventid', 'latitude', 'longitude', 'specificity', 
                              'vicinity', 'location', 'summary', 'motive']].dropna(subset=['eventid'])

            place_spec_entries: List[Dict[str, int|bool|float|str]] = [
                {
                    "id": int(row["eventid"]),
                    "latitude": float(row["latitude"]) if pd.notna(row["latitude"]) else None,
                    "longitude": float(row["longitude"]) if pd.notna(row["longitude"]) else None,
                    "specificity": int(row["specificity"]) if pd.notna(row["specificity"]) else None,
                    "vicinity": ORM.clean_boolean(row["vicinity"]), 
                    "location": row["location"] if pd.notna(row["location"]) else None,
                    "summary": row["summary"] if pd.notna(row["summary"]) else None,
                    "motive": row["motive"] if pd.notna(row["motive"]) else None,
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", place_spec_entries[:10])  
            session.bulk_insert_mappings(PlaceSpec, place_spec_entries)
            session.commit()

    @staticmethod
    def fill_attack_spec(): 
        
        """Method. Filling the table attack_spec"""
        
        with Session() as session:
          
            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}
    
            filtered_df: pd.DataFrame = df[['eventid', 'crit1', 'crit2', 'crit3', 
                              'doubtterr', 'multiple', 'success', 'suicide']].dropna(subset=['eventid'])
    
            attack_specc_entries: List[Dict[str, int|bool]] = [
                {
                    "id": int(row["eventid"]),
                    "crit1": ORM.clean_boolean(row["crit1"]),
                    "crit2": ORM.clean_boolean(row["crit2"]),
                    "crit3": ORM.clean_boolean(row["crit3"]),
                    "doubtterr": ORM.clean_boolean(row["doubtterr"]),
                    "multiple": ORM.clean_boolean(row["multiple"]),
                    "success": ORM.clean_boolean(row["success"]),
                    "suicide": ORM.clean_boolean(row["suicide"]),
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", attack_specc_entries[:10]) 
            session.bulk_insert_mappings(AttackSpec, attack_specc_entries)
            session.commit()
            
    @staticmethod
    def clean_boolean(value)->bool:

        """Method. Check boolean values
        Parametrs:
        value -boolean date for check (None or -9)
        Return bool
        """

        if pd.isna(value) or value == -9:
            return False  # Instead of None we put False
        return bool(value)

    @staticmethod
    def fill_victims():    
        
         """Method. Filling the table victims"""
        
        with Session() as session:

            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}
    
            filtered_df: pd.DataFrame = df[['eventid', 'nkill', 'nkillus', 'nkillter', 
                              'nwound', 'nwoundus', 'nwoundte']].dropna(subset=['eventid'])
    
            victims_entries: List[Dict[str, int]] = [
                {
                    "id": int(row["eventid"]),
                    "nkill": int(row["nkill"]) if pd.notna(row["nkill"]) else None,
                    "nkillus": int(row["nkillus"]) if pd.notna(row["nkillus"]) else None,
                    "nkillter": int(row["nkillter"]) if pd.notna(row["nkillter"]) else None,
                    "nwound": int(row["nwound"]) if pd.notna(row["nwound"]) else None,
                    "nwoundus": int(row["nwoundus"]) if pd.notna(row["nwoundus"]) else None,
                    "nwoundte": int(row["nwoundte"]) if pd.notna(row["nwoundte"]) else None,
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", victims_entries[:10])  # Проверка первых 10 элементов
            session.bulk_insert_mappings(Victims, victims_entries)
            session.commit()

    @staticmethod
    def fill_properties(): 
        
        """Method. Filling the table properties"""
        
        with Session() as session:

            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}
    
            filtered_df: pd.DataFrame = df[['eventid', 'property', 'propvalue', 'propcomment']].dropna(subset=['eventid'])
    
            properties_entries: List[Dict[str, int | bool| str]] = [
                {
                    "id": int(row["eventid"]),
                    "property_": ORM.clean_boolean(row["property"]),
                    "propvalue": int(row["propvalue"]) if pd.notna(row["propvalue"]) and row["propvalue"] != -9 else 0,
                    "propcomment": row["propcomment"] if pd.notna(row["propcomment"]) else "",
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", properties_entries[:10])  
            session.bulk_insert_mappings(Properties, properties_entries)
            session.commit()

    @staticmethod
    def fill_hostages():     
        
        """Method. Filling the table hostages"""
        
        with Session() as session:

            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}

            filtered_df: pd.DataFrame = df[['eventid', 'ishostkid', 'nhostkid', 'nhostkidus', 'ndays', 'ransom', 'ransomamt', 'ransompaid', 'hostkidoutcome_txt']].dropna(subset=['eventid'])
    
            hostages_entries: List[Dict[str, int | bool]]  = [
                {
                    "id": int(row["eventid"]),
                    "ishostkid": ORM.clean_boolean(row["ishostkid"]),
                    "nhostkid": int(row["nhostkid"]) if pd.notna(row["nhostkid"]) else None,
                    "nhostkidus": int(row["nhostkidus"]) if pd.notna(row["nhostkidus"]) else None,
                    "ndays": int(row["ndays"]) if pd.notna(row["ndays"]) else None,
                    "ransom": ORM.clean_boolean(row["ransom"]),
                    "ransomamt": int(row["ransomamt"]) if pd.notna(row["ransomamt"]) else None,
                    "ransompaid": int(row["ransompaid"]) if pd.notna(row["ransompaid"]) else None,
                    "hostkidoutcome_txt": row["hostkidoutcome_txt"] if pd.notna(row["hostkidoutcome_txt"]) else "",
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", hostages_entries[:10])  
            session.bulk_insert_mappings(Hostages, hostages_entries)
            session.commit()

    @staticmethod
    def fill_international_data():
        
        """Method. Filling the table international_data"""
        
        with Session() as session:

            incident_ids: set[int] = {i.id for i in session.query(Incident.id).all()}

            filtered_df: pd.DataFrame = df[['eventid', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY']].dropna(subset=['eventid'])

            international_data_entries: List[Dict[str, int | bool]]  = [
                {
                    "id": int(row["eventid"]),
                    "INT_LOG": ORM.clean_boolean(row["INT_LOG"]),
                    "INT_IDEO": ORM.clean_boolean(row["INT_IDEO"]),
                    "INT_MISC": ORM.clean_boolean(row["INT_MISC"]),
                    "INT_ANY": ORM.clean_boolean(row["INT_ANY"]),
                    "incident_id": int(row["eventid"])
                }
                for _, row in filtered_df.iterrows()
                if int(row["eventid"]) in incident_ids
            ]
            
            print("Data to insert:", international_data_entries[:10])  #
            session.bulk_insert_mappings(International_Data, international_data_entries)
            session.commit()




In [33]:
# We perform filling (i.e. filling from the data frame - one-time operations)


In [88]:
# ORM.fill_type_incident()

In [89]:
# ORM.fill_countries()

In [90]:
# ORM.fill_regions()

In [91]:
# ORM.fill_cities()

In [92]:
# ORM.fill_type_attack()

In [93]:
# ORM.fill_target_type()

In [94]:
# ORM.fill_target_sub_type()

Данные для вставки: [{'id': 68, 'targsubtype1_txt': 'Named Civilian', 'target_type_id': 14}, {'id': 45, 'targsubtype1_txt': 'Diplomatic Personnel (outside of embassy, consulate)', 'target_type_id': 7}, {'id': 54, 'targsubtype1_txt': 'Radio Journalist/Staff/Facility', 'target_type_id': 10}, {'id': 46, 'targsubtype1_txt': 'Embassy/Consulate', 'target_type_id': 7}, {'id': 22, 'targsubtype1_txt': 'Police Building (headquarters, station, school)', 'target_type_id': 3}, {'id': 25, 'targsubtype1_txt': 'Police Security Forces/Officers', 'target_type_id': 3}, {'id': 107, 'targsubtype1_txt': 'Electricity', 'target_type_id': 21}, {'id': 28, 'targsubtype1_txt': 'Military Recruiting Station/Academy', 'target_type_id': 4}, {'id': 21, 'targsubtype1_txt': 'Government Building/Facility/Office', 'target_type_id': 2}, {'id': 27, 'targsubtype1_txt': 'Military Barracks/Base/Headquarters/Checkpost', 'target_type_id': 4}]


In [95]:
# ORM.fill_corp()

Данные для вставки: [{'corp1': 'Belgian Ambassador Daughter'}, {'corp1': 'Voice of America'}, {'corp1': 'Cairo Police Department'}, {'corp1': 'Uruguayan Police'}, {'corp1': 'Pacific Gas & Electric Company'}, {'corp1': 'R.O.T.C.'}, {'corp1': 'Selective Service'}, {'corp1': 'Army Recruiting Station'}, {'corp1': 'Trans World Airline'}, {'corp1': 'U.S. Government housing'}]


In [96]:
# ORM.fill_target()

Данные для вставки: [{'target1': 'Julio Guzman'}, {'target1': 'Nadine Chaval, daughter'}, {'target1': 'Employee'}, {'target1': 'U.S. Embassy'}, {'target1': 'U.S. Consulate'}, {'target1': 'Cairo Police Headquarters'}, {'target1': 'Juan Maria de Lucah/Chief of Directorate of info. and intell.'}, {'target1': 'Edes Substation'}, {'target1': 'R.O.T.C. offices at University of Wisconsin, Madison'}, {'target1': 'Selective Service Headquarters in Madison Wisconsin'}]


In [97]:
# ORM.fill_natlty()

Данные для вставки: [{'id': 58, 'natlty1_txt': 'Dominican Republic'}, {'id': 21, 'natlty1_txt': 'Belgium'}, {'id': 217, 'natlty1_txt': 'United States'}, {'id': 218, 'natlty1_txt': 'Uruguay'}, {'id': 75, 'natlty1_txt': 'Germany'}, {'id': 216, 'natlty1_txt': 'Great Britain'}, {'id': 222, 'natlty1_txt': 'Venezuela'}, {'id': 163, 'natlty1_txt': 'Puerto Rico'}, {'id': 97, 'natlty1_txt': 'Israel'}, {'id': 15, 'natlty1_txt': 'Austria'}]


In [98]:
# ORM.fill_claimmode()

Данные для вставки: [{'id': 1, 'claimmode_txt': 'Letter'}, {'id': 10, 'claimmode_txt': 'Unknown'}, {'id': 3, 'claimmode_txt': 'Call (pre-incident)'}, {'id': 9, 'claimmode_txt': 'Other'}, {'id': 8, 'claimmode_txt': 'Personal claim'}, {'id': 5, 'claimmode_txt': 'Note left at scene'}, {'id': 2, 'claimmode_txt': 'Call (post-incident)'}, {'id': 7, 'claimmode_txt': 'Posted to website, blog, etc.'}, {'id': 4, 'claimmode_txt': 'E-mail'}, {'id': 6, 'claimmode_txt': 'Video'}]


In [99]:
# ORM.fill_weapon_type()

Данные для вставки: [{'id': 13, 'weaptype1_txt': 'Unknown'}, {'id': 6, 'weaptype1_txt': 'Explosives'}, {'id': 8, 'weaptype1_txt': 'Incendiary'}, {'id': 5, 'weaptype1_txt': 'Firearms'}, {'id': 2, 'weaptype1_txt': 'Chemical'}, {'id': 9, 'weaptype1_txt': 'Melee'}, {'id': 11, 'weaptype1_txt': 'Sabotage Equipment'}, {'id': 10, 'weaptype1_txt': 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)'}, {'id': 7, 'weaptype1_txt': 'Fake Weapons'}, {'id': 3, 'weaptype1_txt': 'Radiological'}]


In [100]:
# ORM.fill_weapon_sub_type()

Данные для вставки: [{'id': 16, 'weapsubtype1_txt': 'Unknown Explosive Type', 'weapon_type_id': 6}, {'id': 5, 'weapsubtype1_txt': 'Unknown Gun Type', 'weapon_type_id': 5}, {'id': 2, 'weapsubtype1_txt': 'Automatic or Semi-Automatic Rifle', 'weapon_type_id': 5}, {'id': 19, 'weapsubtype1_txt': 'Molotov Cocktail/Petrol Bomb', 'weapon_type_id': 8}, {'id': 20, 'weapsubtype1_txt': 'Gasoline or Alcohol', 'weapon_type_id': 8}, {'id': 4, 'weapsubtype1_txt': 'Rifle/Shotgun (non-automatic)', 'weapon_type_id': 5}, {'id': 18, 'weapsubtype1_txt': 'Arson/Fire', 'weapon_type_id': 8}, {'id': 31, 'weapsubtype1_txt': 'Pipe Bomb', 'weapon_type_id': 6}, {'id': 15, 'weapsubtype1_txt': 'Vehicle', 'weapon_type_id': 6}, {'id': 28, 'weapsubtype1_txt': 'Dynamite/TNT', 'weapon_type_id': 6}]


In [101]:
# ORM.fill_propextent()

Данные для вставки: [{'id': 3, 'propextent_txt': 'Minor (likely < $1 million)'}, {'id': 2, 'propextent_txt': 'Major (likely >= $1 million but < $1 billion)'}, {'id': 4, 'propextent_txt': 'Unknown'}, {'id': 1, 'propextent_txt': 'Catastrophic (likely >= $1 billion)'}]


In [102]:
# ORM.fill_incident()

In [103]:
# ORM.fill_other_data()

Данные для вставки: [{'id': 197000000001, 'addnotes': '', 'scite1': '', 'scite2': '', 'scite3': '', 'incident_id': 197000000001}, {'id': 197000000002, 'addnotes': '', 'scite1': '', 'scite2': '', 'scite3': '', 'incident_id': 197000000002}, {'id': 197001000001, 'addnotes': '', 'scite1': '', 'scite2': '', 'scite3': '', 'incident_id': 197001000001}, {'id': 197001000002, 'addnotes': '', 'scite1': '', 'scite2': '', 'scite3': '', 'incident_id': 197001000002}, {'id': 197001000003, 'addnotes': '', 'scite1': '', 'scite2': '', 'scite3': '', 'incident_id': 197001000003}, {'id': 197001010002, 'addnotes': 'The Cairo Chief of Police, William Petersen, resigned as a result of the attack.', 'scite1': '"Police Chief Quits," Washington Post, January 2, 1970.', 'scite2': '"Cairo Police Chief Quits; Decries Local \'Militants\'," Afro-American, January 10, 1970.', 'scite3': 'Christopher Hewitt, "Political Violence and Terrorism in Modern America: A Chronology," Praeger Security International, 2005.', 'incid

In [104]:
# ORM.fill_place_spec()

Данные для вставки: [{'id': 197000000001, 'latitude': 18.456792, 'longitude': -69.951164, 'specificity': 1, 'vicinity': False, 'location': None, 'summary': None, 'motive': None, 'incident_id': 197000000001}, {'id': 197000000002, 'latitude': 19.371887, 'longitude': -99.086624, 'specificity': 1, 'vicinity': False, 'location': None, 'summary': None, 'motive': None, 'incident_id': 197000000002}, {'id': 197001000001, 'latitude': 15.478598, 'longitude': 120.599741, 'specificity': 4, 'vicinity': False, 'location': None, 'summary': None, 'motive': None, 'incident_id': 197001000001}, {'id': 197001000002, 'latitude': 37.99749, 'longitude': 23.762728, 'specificity': 1, 'vicinity': False, 'location': None, 'summary': None, 'motive': None, 'incident_id': 197001000002}, {'id': 197001000003, 'latitude': 33.580412, 'longitude': 130.396361, 'specificity': 1, 'vicinity': False, 'location': None, 'summary': None, 'motive': None, 'incident_id': 197001000003}, {'id': 197001010002, 'latitude': 37.005105, 'l

In [105]:
# ORM.fill_attack_spec()

Данные для вставки: [{'id': 197000000001, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicide': False, 'incident_id': 197000000001}, {'id': 197000000002, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicide': False, 'incident_id': 197000000002}, {'id': 197001000001, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicide': False, 'incident_id': 197001000001}, {'id': 197001000002, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicide': False, 'incident_id': 197001000002}, {'id': 197001000003, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicide': False, 'incident_id': 197001000003}, {'id': 197001010002, 'crit1': True, 'crit2': True, 'crit3': True, 'doubtterr': False, 'multiple': False, 'success': True, 'suicid

In [106]:
# ORM.fill_victims()

Данные для вставки: [{'id': 197000000001, 'nkill': 1, 'nkillus': None, 'nkillter': None, 'nwound': 0, 'nwoundus': None, 'nwoundte': None, 'incident_id': 197000000001}, {'id': 197000000002, 'nkill': 0, 'nkillus': None, 'nkillter': None, 'nwound': 0, 'nwoundus': None, 'nwoundte': None, 'incident_id': 197000000002}, {'id': 197001000001, 'nkill': 1, 'nkillus': None, 'nkillter': None, 'nwound': 0, 'nwoundus': None, 'nwoundte': None, 'incident_id': 197001000001}, {'id': 197001000002, 'nkill': None, 'nkillus': None, 'nkillter': None, 'nwound': None, 'nwoundus': None, 'nwoundte': None, 'incident_id': 197001000002}, {'id': 197001000003, 'nkill': None, 'nkillus': None, 'nkillter': None, 'nwound': None, 'nwoundus': None, 'nwoundte': None, 'incident_id': 197001000003}, {'id': 197001010002, 'nkill': 0, 'nkillus': 0, 'nkillter': 0, 'nwound': 0, 'nwoundus': 0, 'nwoundte': 0, 'incident_id': 197001010002}, {'id': 197001020001, 'nkill': 0, 'nkillus': None, 'nkillter': None, 'nwound': 0, 'nwoundus': None

In [107]:
# ORM.fill_properties()

Данные для вставки: [{'id': 197000000001, 'property_': False, 'propvalue': 0, 'propcomment': '', 'incident_id': 197000000001}, {'id': 197000000002, 'property_': False, 'propvalue': 0, 'propcomment': '', 'incident_id': 197000000002}, {'id': 197001000001, 'property_': False, 'propvalue': 0, 'propcomment': '', 'incident_id': 197001000001}, {'id': 197001000002, 'property_': True, 'propvalue': 0, 'propcomment': '', 'incident_id': 197001000002}, {'id': 197001000003, 'property_': True, 'propvalue': 0, 'propcomment': '', 'incident_id': 197001000003}, {'id': 197001010002, 'property_': True, 'propvalue': 0, 'propcomment': '', 'incident_id': 197001010002}, {'id': 197001020001, 'property_': False, 'propvalue': 0, 'propcomment': '', 'incident_id': 197001020001}, {'id': 197001020002, 'property_': True, 'propvalue': 22500, 'propcomment': 'Three transformers were damaged.', 'incident_id': 197001020002}, {'id': 197001020003, 'property_': True, 'propvalue': 60000, 'propcomment': 'Basketball courts, weig

In [108]:
# ORM.fill_hostages()

Данные для вставки: [{'id': 197000000001, 'ishostkid': False, 'nhostkid': None, 'nhostkidus': None, 'ndays': None, 'ransom': False, 'ransomamt': None, 'ransompaid': None, 'hostkidoutcome_txt': '', 'incident_id': 197000000001}, {'id': 197000000002, 'ishostkid': True, 'nhostkid': 1, 'nhostkidus': 0, 'ndays': None, 'ransom': True, 'ransomamt': 800000, 'ransompaid': None, 'hostkidoutcome_txt': '', 'incident_id': 197000000002}, {'id': 197001000001, 'ishostkid': False, 'nhostkid': None, 'nhostkidus': None, 'ndays': None, 'ransom': False, 'ransomamt': None, 'ransompaid': None, 'hostkidoutcome_txt': '', 'incident_id': 197001000001}, {'id': 197001000002, 'ishostkid': False, 'nhostkid': None, 'nhostkidus': None, 'ndays': None, 'ransom': False, 'ransomamt': None, 'ransompaid': None, 'hostkidoutcome_txt': '', 'incident_id': 197001000002}, {'id': 197001000003, 'ishostkid': False, 'nhostkid': None, 'nhostkidus': None, 'ndays': None, 'ransom': False, 'ransomamt': None, 'ransompaid': None, 'hostkidout

In [109]:
# ORM.fill_international_data()

Данные для вставки: [{'id': 197000000001, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': False, 'INT_ANY': False, 'incident_id': 197000000001}, {'id': 197000000002, 'INT_LOG': False, 'INT_IDEO': True, 'INT_MISC': True, 'INT_ANY': True, 'incident_id': 197000000002}, {'id': 197001000001, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': True, 'INT_ANY': True, 'incident_id': 197001000001}, {'id': 197001000002, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': True, 'INT_ANY': True, 'incident_id': 197001000002}, {'id': 197001000003, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': True, 'INT_ANY': True, 'incident_id': 197001000003}, {'id': 197001010002, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': False, 'INT_ANY': False, 'incident_id': 197001010002}, {'id': 197001020001, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': False, 'INT_ANY': False, 'incident_id': 197001020001}, {'id': 197001020002, 'INT_LOG': False, 'INT_IDEO': False, 'INT_MISC': False, 'INT_ANY': False, 'incident_id': 197

# Make graf

In [118]:
import plotly.graph_objs as go
from sqlalchemy.orm import joinedload
from collections import defaultdict
import plotly.io as pio
pio.renderers.default = "iframe"

## Graf 1. Terror and non-terror incident

In [120]:
Session = sessionmaker(bind=engine)

with Session() as session:
    results = session.query(
        Incident.iyear,
        AttackSpec.crit1,
        AttackSpec.crit2,
        AttackSpec.crit3
    ).join(AttackSpec, AttackSpec.incident_id == Incident.id).all()

# Processing data
data = defaultdict(lambda: {"terrorist": 0, "non_terrorist": 0})

for year, crit1, crit2, crit3 in results:
    if crit1 and crit2 and crit3:
        data[year]["terrorist"] += 1
    else:
        data[year]["non_terrorist"] += 1

# Convert to DataFrame
df_graf = pd.DataFrame([
    {"Year": year, "Incidents": count, "Type": label}
    for year, counts in data.items()
    for label, count in counts.items()
]).pivot(index="Year", columns="Type", values="Incidents").fillna(0)

# Make 2 lines
trace1 = go.Scatter(
    x=df_graf.index.tolist(), 
    y=df_graf["terrorist"].tolist(), 
    mode="lines+markers",
    name="Terrorist Incidents",
    marker=dict(color='red')
)

trace2 = go.Scatter(
    x=df_graf.index.tolist(), 
    y=df_graf["non_terrorist"].tolist(), 
    mode="lines+markers",
    name="Other Incidents",
    marker=dict(color='blue')
)

# make graf
layout = dict(
    title="Number of Incidents per Year",
    xaxis=dict(title="Year", ticklen=5, zeroline=False, type="category"),
    yaxis=dict(title="Number of Incidents"),
    hovermode="x"
)

# show graf
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show(renderer="iframe")

## Graf 2. Victims and Hostage

In [121]:

Session = sessionmaker(bind=engine)

with Session() as session:
    results = session.query(
        Incident.iyear,
        Victims.nkill,
        Hostages.nhostkid
    ).outerjoin(Victims, Victims.incident_id == Incident.id)\
     .outerjoin(Hostages, Hostages.incident_id == Incident.id)\
     .all()


data = defaultdict(lambda: {"killed": 0, "hostages": 0})

for year, nkill, nhostkid in results:
    if nkill:
        data[year]["killed"] += nkill
    if nhostkid:
        data[year]["hostages"] += nhostkid


df = pd.DataFrame([
    {"Year": year, "Killed": counts["killed"], "Hostages": counts["hostages"]}
    for year, counts in data.items()
]).set_index("Year").sort_index()

# make graf
trace1 = go.Bar(x=df.index, y=df["Killed"], name="Killed", marker=dict(color='red'))
trace2 = go.Bar(x=df.index, y=df["Hostages"], name="Hostages", marker=dict(color='blue'))

layout = dict(
    title="Killed vs Hostages per Year",
    xaxis=dict(title="Year"),
    yaxis=dict(title="Number of People"),
    barmode='overlay',  # Overlay columns
    hovermode="x"
)

# show graf
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show(renderer="iframe") 

## Graf 3. Incidents by country

In [124]:
Session = sessionmaker(bind=engine)

with Session() as session:
    results = session.query(
        Incident.iyear,
        City.city,
        Region.region_txt,
        Country.country_txt 
    ).join(City, City.id == Incident.city_id)\
     .join(Region, Region.id == City.region_id)\
     .join(Country, Country.id == Region.country_id)\
     .all()

data = defaultdict(int)

for _, _, _, country in results:
    data[country] += 1 # We count the number of incidents by country

df = pd.DataFrame(list(data.items()), columns=["Country", "Incidents"])

# Sort and take the top 10 countries (if there is too much data)
df = df.sort_values(by="Incidents", ascending=False).head(10)

# Create a pie chart
fig = go.Figure(data=[go.Pie(labels=df["Country"], values=df["Incidents"], hole=0.3)])

fig.update_layout(
    title="Top 10 Countries by Number of Incidents",
)

fig.show(renderer="iframe") 


